<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio EDA y Gestión de Nulos

AutoRenew, una empresa líder en la venta de coches de segunda mano, te ha contratado para desarrollar un modelo predictivo que estime el precio de los vehículos usados. Este modelo ayudará a la empresa a mejorar sus estrategias de ventas, marketing y optimización del inventario. Para ello, cuentas con un conjunto de datos que contiene información detallada sobre diversas características de los coches usados. Sin embargo, como es común en conjuntos de datos reales, encontrarás valores nulos y necesitarás realizar un análisis exploratorio para identificar patrones, correlaciones y posibles anomalías.

 En este laboratorio, tu tarea es realizar un análisis exploratorio de los datos (EDA) y gestionar los valores nulos que encuentres. Este es solo el primer paso de un proyecto que se desarrollará durante toda la semana, en el que usarás este mismo conjunto de datos para practicar diferentes fases del preprocesamiento.

**Columnas del conjunto de datos:**

- **dateCrawled**: Fecha y hora en que se recolectaron los datos del coche.

- **name**: Nombre del anuncio, generalmente incluye la marca y el modelo del coche.

- **seller**: Tipo de vendedor (particular o concesionario).

- **offerType**: Tipo de oferta (venta o subasta).

- **price**: Precio del coche en la moneda local.

- **abtest**: Campo utilizado para pruebas A/B, indicando diferentes versiones del sitio web o anuncio.

- **vehicleType**: Tipo de vehículo (sedán, SUV, hatchback, etc.).

- **yearOfRegistration**: Año de registro del coche.

- **gearbox**: Tipo de caja de cambios (manual o automática).

- **powerCV**: Potencia del coche en caballos de vapor (CV).

- **model**: Modelo específico del coche.

- **kilometer**: Kilometraje total del coche.

- **monthOfRegistration**: Mes de registro del coche.

- **fuelType**: Tipo de combustible (gasolina, diésel, eléctrico, etc.).

- **brand**: Marca del coche.

- **notRepairedDamage**: Indica si el coche tiene daños no reparados.

- **dateCreated**: Fecha de creación del anuncio.

- **nrOfPictures**: Número de fotos en el anuncio.

- **postalCode**: Código postal de la ubicación del coche.

- **lastSeen**: Última fecha y hora en que el anuncio fue visto.

**Instrucciones:**

1. **Análisis Exploratorio de Datos (EDA):** Explora el conjunto de datos para familiarizarte con las diferentes características de los coches. Utiliza gráficos y estadísticas descriptivas para identificar patrones, tendencias y posibles anomalías.
   
2. **Gestión de Valores Nulos:** Identifica las columnas que contienen valores nulos y aplica las técnicas adecuadas para manejarlos. Esto podría incluir la imputación de valores, eliminación de filas o columnas, o cualquier otra estrategia que consideres apropiada.

**Nota:** Este laboratorio es el primer paso de una serie de actividades que realizarás esta semana utilizando el mismo conjunto de datos. Asegúrate de documentar bien tu trabajo, ya que lo utilizarás como base para los siguientes ejercicios.


In [2]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math
import numpy as np
import sys
sys.path.append('src')  # Agrega la carpeta 'src' al PATH

from src import support as sp  # Importa la función desde el archivo

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor


In [3]:
df=pd.read_csv("data/autorenew.csv", index_col=0)

In [4]:
df.sample(5)

,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerCV,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
dateCrawled,,,,,,,,,,,,,,,,,,,
2016-03-20 18:38:20,Vw_golf_1_cabrio,private,offer,0,test,convertible,1993,manually,97.0,golf,125000,0,NaN,volkswagen,NaN,2016-03-20 00:00:00,0,88299,2016-04-07 01:45:08
2016-04-03 18:40:34,Ford_Fiesta_1.4_TDCI_Fun,private,offer,2900,control,small car,2006,NaN,67.0,fiesta,150000,12,diesel,ford,no,2016-04-03 00:00:00,0,55232,2016-04-03 18:40:34
2016-03-27 22:53:44,Opel_Signum_3.2_V6_Cosmo,private,offer,2250,test,combi,2003,manually,208.0,signum,150000,4,petrol,opel,no,2016-03-27 00:00:00,0,73540,2016-04-06 01:15:28
2016-03-12 07:55:15,Astrabf_cabrio__fahrbereit__6/16tuv,private,offer,300,control,convertible,1996,manually,70.0,astra,150000,0,NaN,opel,no,2016-03-12 00:00:00,0,34132,2016-03-25 02:47:10
2016-03-16 21:47:16,Opel_Vectra_B_Kombi,private,offer,400,control,combi,1998,manually,89.0,vectra,150000,0,petrol,opel,no,2016-03-16 00:00:00,0,26219,2016-04-07 09:45:45


In [5]:
df.describe()

,price,yearOfRegistration,powerCV,kilometer,monthOfRegistration,nrOfPictures,postalCode
count,3.715280e+05,371528.000000,371528.000000,371528.000000,371528.000000,371528.0,371528.00000
mean,1.729514e+04,2004.577997,113.967833,125618.688228,5.734445,0.0,50820.66764
std,3.587954e+06,92.866598,189.516594,40112.337051,3.712412,0.0,25799.08247
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.00000
25%,1.150000e+03,1999.000000,69.000000,125000.000000,3.000000,0.0,30459.00000
50%,2.950000e+03,2003.000000,104.000000,150000.000000,6.000000,0.0,49610.00000
75%,7.200000e+03,2008.000000,148.000000,150000.000000,9.000000,0.0,71546.00000
max,2.147484e+09,9999.000000,19726.000000,150000.000000,12.000000,0.0,99998.00000


In [6]:
df.shape

(371528, 19)

In [7]:
df.nunique()

name                   233531
seller                      2
offerType                   2
price                    5597
abtest                      2
vehicleType                 7
yearOfRegistration        155
gearbox                     2
powerCV                   788
model                     251
kilometer                  13
monthOfRegistration        13
fuelType                    7
brand                      40
notRepairedDamage           2
dateCreated               114
nrOfPictures                1
postalCode               8150
lastSeen               182806
dtype: int64

In [8]:
df.isnull().sum()

name                        0
seller                      0
offerType                   0
price                       0
abtest                      0
vehicleType            133763
yearOfRegistration          0
gearbox                 20209
powerCV                     0
model                   20484
kilometer                   0
monthOfRegistration         0
fuelType                33386
brand                       0
notRepairedDamage       72060
dateCreated                 0
nrOfPictures                0
postalCode                  0
lastSeen                    0
dtype: int64

In [9]:
sp.exploracion_basica_dataframe(df)

El número de datos es 371528 y el de columnas es 19

 ------------------------------- 

Los duplicados que tenemos en el conjunto de datos son: 29

 ------------------------------- 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos
vehicleType,36.003478
gearbox,5.439429
model,5.513447
fuelType,8.986133
notRepairedDamage,19.395577



 ------------------------------- 

Los tipos de las columnas son:


,tipo_dato
name,object
seller,object
offerType,object
price,int64
abtest,object
vehicleType,object
yearOfRegistration,int64
gearbox,object
powerCV,float64
model,object



 ------------------------------- 

Los valores que tenemos para las columnas categóricas son: 
Index(['name', 'seller', 'offerType', 'abtest', 'vehicleType', 'gearbox',
       'model', 'fuelType', 'brand', 'notRepairedDamage', 'dateCreated',
       'lastSeen'],
      dtype='object')
La columna NAME tiene las siguientes valore únicos:


...

,count
monthOfRegistration,
0,37675
3,36170
6,33167
4,30918
5,30631
7,28958
10,27337
11,25489
12,25380


La columna NROFPICTURES tiene las siguientes valore únicos:


,count
nrOfPictures,
0,371528


La columna POSTALCODE tiene las siguientes valore únicos:


,count
postalCode,
10115,828
65428,637
66333,349
38518,326
44145,323
...,...
94167,1
25599,1
79697,1


In [ ]:
# Ver valores atipicos de "yearOfRegistration"
limite_inferior = 1885
limite_superior = 2024
valores_fuera_rango = df[(df['yearOfRegistration'] < limite_inferior) | (df['yearOfRegistration'] > limite_superior)]
valores_fuera_rango.shape

(182, 19)